# Likelihoods for Retro

For Retro Reco, we need to define an appropriate likelihood function in order to perform maximum-likelihood estimation of an event's parameters.

Here, I'll write out one function that seems appropriate to use which includes as much information as possible (and hence should minimize the variance in our ML estimates).

Subsequently, I'll describe how we can evaluate this in the context of likelihood maximization.

## Definitions

* $L$ : likelihood
* ${\mathscr L}\equiv\ln L$ : log likelihood

* $d$ : sensor index
* $D_{\rm tot}$ : total number of operational sensors
* $D_{\rm hit}$ : number of hit sensors

* $k_d$ : hit index for sensor $d$
* $K_d$ : total number of hits on sensor $d$
* $t_{k_d}$ : time of hit $k_d$ on sensor $d$
* $q_{k_d}$ : detected charge for hit $k_d$ in sensor $d$
* $Q_d \equiv \sum_{k_d=1}^{K_d} q_{k_d}$ : total charge detected in sensor $d$

* $\lambda_d = \lambda_d(t)$ : signal-charge rate expectation in sensor $d$ as a function of time; depends upon the hypothesis; units of charge/time
* $\Lambda_d \equiv \int_{\rm time \, window} \lambda_{d}(t)\;{\rm d}t $ : total signal-charge expectation in sensor $d$; depends upon the hypothesis; units of charge
* $\Lambda \equiv \sum_{d=1}^{D_{\rm tot}} \Lambda_d $ : total signal-charge expectation for _entire_ detector; depends upon the hypothesis; units of charge

* $n_d$ : noise-charge rate for sensor $d$; units of charge/time
* $N_d \equiv \int_{\rm time \, window} n_d \, {\rm d}t$ : total noise charge expected to be detected in sensor $d$ in the time window; units of charge
* $N \equiv \sum_{d=1}^{D_{\rm tot}} N_d$ : total noise charge expected for all operational sensors in the time window; units of charge

## The extended likelihood function

We use the *_extended likelihood function_* from [_Statistical Data Analysis_, G. Cowan, 1998](https://www.amazon.com/dp/0198501552/?tag=mh0b-20&hvadid=4162881975&hvqmt=b&hvbmt=bb&hvdev=c&ref=pd_sl_6sb6dhwzqt_b) or–for a more precise explanation, two illuminating derivations, better view of why/where to use extended maximum likelihood (and that contradicts Cowan on where extended maximum likelihood outperforms ML)–[_Extended maximum likelihood_, R. Barlow](https://www.sciencedirect.com/science/article/pii/0168900290913348).

The extended likelihood function is defined to include both the probability of detecting charges $\{q_{k_d}\}$ at times $\{t_{k_d}\}$ in sensors $\{d\}$ having expected charge at those times $\lambda_d(t_{k_d})$ _and_ the probability that the sensor saw a total charge of $Q_d$ while one would expect a total of $\Lambda_d$ to be detected in that sensor given the time-window of observation.

The former accounts for _detected charge_ (and when it arrived), while the latter accounts for _missing charge_, given the hypothesis.
Note that these two things show up in opposite order in the equation:

$$L = \prod_{d=1}^{D_{\rm tot}} \left( P(Q_d \, | \, \Lambda_d) \prod_{k_d = 1}^{K_d} {\rm pdf} \left( q_{k_d} \, | \, \lambda_d(t_{k_d}) \right) \right) $$

The probability $P$ on the left is governed by the continuous-charge analog of Poisson statistics

$$P(Q_d \, | \, \Lambda_d, \, N_d) = \frac{(\Lambda_d + N_d)^{Q_d} e^{-(\Lambda_d + N_d)}}{\Gamma(Q_d + 1)}$$

while the pdf on the right is a probability density that we'll formulate as a function of time due to _both_ the signal _and_ background expectations.

The signal pdf is

$${\rm pdf}_{\lambda_d}(t) = \frac{\lambda_d(t)}{\Lambda_d}$$

while the noise pdf is

$${\rm pdf}_{n_d} = \frac{n_d}{N_d}.$$

Combining these with the appropriate ratio (normalized to keep the integral equaling one in the time window of observation)

$${\rm pdf}(t) = \frac{\Lambda_d}{\Lambda_d + N_d} \cdot \frac{\lambda_d(t)}{\Lambda_d} + \frac{N_d}{\Lambda_d + N_d} \cdot \frac{n_d}{N_d}$$

and simplifying,

$${\rm pdf}(t) = \frac{\lambda_d(t) + n_d}{\Lambda_d + N_d}.$$

This could also be arrived at by defining the total expected charge as a function of time

$$c(t) = \lambda_d(t) + n_d$$

and then converting this into a pdf within the time window

$${\rm pdf}(t) = \frac{c(t)}{\int_{\rm time\,window}c(t)\,{\rm d}t} = \frac{\lambda_d(t) + n_d}{\Lambda_d + N_d}$$

which agrees with the expression above.

Note that since each hit's "charge" is interpreted as a multiplicity of single photoelectrons being detected (i.e., multiple single events), the above expression is raised to the power of the detected charge in subsequent expressions.
This differs from the references cited, as they assume a detection either doesn't occur (count = 0) or does occur (count = 1) and you can always subdivide the time window finely enough to obtain either 0 or 1 counts in each subdivision.
Therefore, in those references, $q_{k_d} = 1$ and $Q_d = K_d$.

Incorporating the above expressions for $P$ and ${\rm pdf}(t)$, the extended likelihood function for us is

$$ L = \prod_{d=1}^{D_{\rm tot}} \left( \frac{(\Lambda_d + N_d)^{Q_d} e^{-(\Lambda_d + N_d)}}{\Gamma(Q_d + 1)} \prod_{k_d = 1}^{K_d} \left( \frac{\lambda_d(t_{k_d}) + n_d}{\Lambda_d + N_d} \right)^{q_{k_d}} \right). $$

Taking the logarithm of both sides and dropping the gamma function (which is constant and so does not affect maximization of the likelihood),

$$ {\mathscr L} = 
    \sum_{d=1}^{D_{\rm tot}}
    \left(
        Q_d \ln (\Lambda_d + N_d) -(\Lambda_d + N_d)
        + \sum_{k_d = 1}^{K_d}
            q_{k_d} \ln
            \left(
                \frac{\lambda_d(t_{k_d}) + n_d}{\Lambda_d + N_d}
            \right)
    \right) $$

Distributing the sums to each term,

$$
{\mathscr L} = 
    \sum_{d=1}^{D_{\rm tot}}
        Q_d \ln (\Lambda_d + N_d)
    -\sum_{d=1}^{D_{\rm tot}}
        (\Lambda_d + N_d)
    +\sum_{d=1}^{D_{\rm tot}}
        \sum_{k_d = 1}^{K_d}
            q_{k_d} \ln
            \left(
                \lambda_d(t_{k_d}) + n_d
            \right)
    -\sum_{d=1}^{D_{\rm tot}}
        \sum_{k_d = 1}^{K_d}
            q_{k_d} \ln
            \left(
                \Lambda_d + N_d
            \right)
$$

and it can be seen that the first and fourth terms are equal but have opposite signs, the second term can be replaced by "total expectations" definitions, and the outer sum on in the third term need only run over sensors that actually detected charge during the event:

$$
{\mathscr L} = 
    -(\Lambda + N)
    +\sum_{d=1}^{D_{\rm hit}}
        \sum_{k_d = 1}^{K_d}
            q_{k_d} \ln
            \left(
                \lambda_d(t_{k_d}) + n_d
            \right).
$$

The final step is to realize that $N$ (the total charge expected from noise in all sensors) is independent of the hypothesis and so does not affect the location of the maximum of $\mathscr L$, leaving the following to be maximized:

$$
{\mathscr L} = 
    -\Lambda
    +\sum_{d=1}^{D_{\rm hit}}
        \sum_{k_d = 1}^{K_d}
            q_{k_d} \ln
            \left(
                \lambda_d(t_{k_d}) + n_d
            \right).
$$

## Finding charge expectations

We now show how to obtain $\Lambda$ and $\lambda_d(t)$, as seen in the expression for $\mathscr L$ above.

This involves the Retro tables (which give photon survival probabilities as functions of several dimensions) and the hypothesis (which gives the coordinates and number of photons of light sources).

### Definitions

* $i$ : light source index
* $\ell_i$ : luminosity—number of photons produced—by light source $i$
* $C_i$ : relevant coordinate of light source $i$
* ${\rm TDI}$ : time- and sensor-(DOM-)independent table
* ${\rm T}_d$ : time-_dependent_ table for sensor $d$

### Equations for charge expectations

The total detected-charge expectation for the entire detector uses the TDI table

$$\Lambda = \sum_{i} \ell_i \cdot {\rm TDI}(C_i)$$

while the detected-charge expectation at the time $t_{k_d}$ of hit $k_d$ in sensor $d$ uses the time-depdendent table specific to sensor $d$

$$\lambda_d(t_{k_d}) = \sum_{i} \ell_i \cdot {\rm T}_d(C_i, t_{k_d}).$$

### Scaling, pegleg, and generic light sources

The actual software implementation for maximizing the likelihood can be made more efficient by separating the light sources into three categories:

* $\ell^s$ : **scaling sources**: The set of light sources for which _only_ luminosity changes with energy; neither the total number nor coordinates of the sources change with energy. Simple cascades can be handled this way with minimal impact on modeling accuracy (within some "reasonable" range of energies where the topology does not change much).
* $\ell^p$ : **pegleg sources**: Increasing energy merely adds light sources to the hypothesis, not affecting the existing sources (either luminosities or coordinates). Simple charged-lepton tracks (e.g. minimum-ionizing $\mu$) can be modeled this way.
* $\ell^g$ : **generic sources**: Any change in any hypothesis parameter affects the quantity, luminosity, and/or coordinates of sources in ways that cannot be included in the above two categories.

Since scaling and pegleg sources are handled separately from generic sources, the generic minimizer (e.g., MultiNest) has to cope with two fewer dimensions than it otherwise would.

#### Scaling sources

We define the scaling sources as a nominal topology of light sources (which won't change) at a nominal energy (which will change):

$$E_{\rm cscd}^{\rm nominal} \mapsto \{ \ell_i^s \}_{i \, \in \, {\rm scaling}}.$$

A reasonably-similar cascade energy $E_{\rm cscd}$ can be modeled by simply scaling the nominal sources' luminosities by a factor $\alpha_s$:

$$E_{\rm cscd} = \alpha_s \cdot E_{\rm cscd}^{\rm nominal} \mapsto \{ \alpha_s \cdot \ell_i^s \}_{i \, \in \, {\rm scaling}}.$$

#### Pegleg sources

Increasing the energy of a pegleg source utilizes all sources (both topology and liminosities) from a previous step and merely adds more sources.

In other words, if

$$ \Lambda^p(E_{\mu,0}) = \sum_{i=1}^{I_{p,0}} \ell^p_i \cdot {\rm TDI}(C_i) $$

and $E_{\mu,1} > E_{\mu,0}$, then

$$ \Lambda^p(E_{\mu,1}) = \Lambda^p(E_{\mu,0}) + \sum_{i=I_{p,0} + 1}^{I_{p,1}} \ell^p_i \cdot {\rm TDI}(C_i) $$

(and analagously for the time-dependent expectations).

#### Final expressions for $\Lambda$ and $\lambda$

We can use these to expand the above expressions for time- and sensor-independent detected-charge expectation to be

$$\Lambda = \alpha_s \cdot \sum_{i = 1}^{I_s} \ell^s_i \cdot {\rm TDI}(C_i) + \sum_{i = 1}^{I_p} \ell^p_i \cdot {\rm TDI}(C_i) + \sum_{i = 1}^{I_g} \ell^g_i \cdot {\rm TDI}(C_i)$$

where superscript $s$ is for "scaling," $p$ is for "pegleg," and $g$ is for "generic."
For time-dependent detected-charge expectation to be

$$\lambda_d(t_{k_d}) = \alpha_s \cdot \sum_{i =1}^{I_s} \ell^s_i \cdot {\rm T}_d(C_i, t_{k_d}) + \sum_{i = 1}^{I_p} \ell^p_i \cdot {\rm T}_d(C_i, t_{k_d}) + \sum_{i = 1}^{I_g} \ell^g_i \cdot {\rm T}_d(C_i, t_{k_d}).$$

#### Full expression for $\mathscr L$

$$
{\mathscr L} =
    -\left( \alpha_s \cdot \sum_{i = 1}^{I_s} \ell^s_i \cdot {\rm TDI}(C_i) + \sum_{i = 1}^{I_p} \ell^p_i \cdot {\rm TDI}(C_i) + \sum_{i = 1}^{I_g} \ell^g_i \cdot {\rm TDI}(C_i) \right)
    +\sum_{d=1}^{D_{\rm hit}}
        \sum_{k_d = 1}^{K_d}
            q_{k_d} \ln
            \left(
                n_d + \alpha_s \cdot \sum_{i =1}^{I_s} \ell^s_i \cdot {\rm T}_d(C_i, t_{k_d}) + \sum_{i = 1}^{I_p} \ell^p_i \cdot {\rm T}_d(C_i, t_{k_d}) + \sum_{i = 1}^{I_g} \ell^g_i \cdot {\rm T}_d(C_i, t_{k_d})
            \right).
$$

## Maximizing $\mathscr L$

### Hypothesis parameters

For maximizing $\mathscr L$, we vary our hypothesis parameters ${\vec \theta} \equiv [\theta_1, \, \theta_2, \, \ldots, \, \theta_j, \, \ldots, \, \theta_J]$ where our 10d hypothesis uses parameters ${\vec \theta} = [x, \, y, \, z, \, t, \, \theta_{\mu}, \, \phi_{\mu}, \, \theta_{\rm cascade}, \, \phi_{\rm cascade}, \, E_{\rm cascade}, \, E_{\mu}]$ and our 8d hypothesis omits cascade direction angles (fixing these to be the same as the muon direction angles).

### Overview of procedure

Maximizing $\mathscr L$ can be done strategically for the three source types listed above to take advantage of performance optimizations particular to each.
For this, we use the following procedure:

* Start with a generic minimizer that sets the hypothesis parameters that only affect the generic light sources: 6 (8) for 8d (10d) hypo. That minimizer requests the value of $-\mathscr L$ for those parameters
    * Enter the pegleg loop, which starts with some small value of the pegleg parameter and produces the corresponding pegleg sources; we will add more sources to these each time around the loop until a stopping condition on the change of $-\mathscr L$ and/or some upper limit of the pegleg parameter is met. At each step:
        * Perform the scaling-factor computation, which starts by creating a nominal topology of sources and their luminosities for the nominal value of the scaling parameter. Then solve numerically for the optimal scaling parameter that minimizes $-\mathscr L$ (given the other 7 (9) parameters. Once this is found, the value of the scaling parameter is set.
    * Once the optimal value is found for the pegleg parameter (number of pegleg sources), this and all other parameters are now set. Return the value of $-\mathscr L$ to the generic minimizer corresponding to the 8 (10) parameter values.

### Jacobian

It will be useful to have an expression for the Jacobain of the scalar function $\mathscr L$ with respect to the vector of $J$ hypothesis parameters, $\{\theta_j\}$ for both examining where extrema occur (by setting this equal to zero) and for performing gradient descent on $-\mathscr L$.
The Jacobain is

$$
\frac{\partial_j {\mathscr L}}{\partial \theta_j} = 
    -\frac{\partial_j {\Lambda}}{\partial \theta_j}
    +\sum_{d=1}^{D_{\rm hit}}
        \sum_{k_d = 1}^{K_d}
            \left(
                \frac{
                    q_{k_d}
                }{
                    \lambda_d(t_{k_d}) + n_d
                }
                \cdot
                \frac{\partial_j {\lambda_d(t_{k_d})}}{\partial \theta_j}
            \right)
$$

where we note that derivatives of $N$ and $n_d$ are both zero since we take the hypothesized noise in each sensor to be known and constant.

Extrema of $\mathscr L$ occur where this is equal to 0, or

$$
    \frac{\partial_j {\Lambda}}{\partial \theta_j}
    =
    \sum_{d=1}^{D_{\rm hit}}
        \sum_{k_d = 1}^{K_d}
            \left(
                \frac{
                    q_{k_d}
                }{
                    \lambda_d(t_{k_d}) + n_d
                }
                \cdot
                \frac{\partial_j {\lambda_d(t_{k_d})}}{\partial \theta_j}
            \right) \; \forall \; j.
$$

### Outermost operation: Generic parameters

Here, the generic minimizer routine picks out some set of the 6 (8) parameters that are not handled by pegleg or scaling routines, and possibly that specify generic sources (in our case, no sources are actually exclusive to generic parameters, all sources require the generic parameters and either the pegleg or scaling parameter).
Based on the values of these parameters, photon sources are generated with accompanying luminosities and then the expectation values
$$\Lambda^g$$
and
$$\lambda_d^g(t_{k_d})$$
can be computed and stored for subsequent operations.
Note that the latter must be stored for each hit time $t_{k_d}$ recorded, while the former is a single value.

The generic minimizer then will request a value of the metric given those parameter values.
To obtain this, we will perform the full pegleg and—within this—scaling procedures to find optimal values of the pegleg and scaling paramters and only after this evaluate the full likelihood function with the combination of all 8 (10) parameters.
This is then the value we will return to the generic minimizer.

### Middle operation: Set the pegleg parameter

Here we start by picking a value for $E_\mu$ that we know to be smaller than its optimal value.

### Innermost operation: Set the scaling parameter

In the procedure outlined above, the innermost operation handles the scaling sources (as we have implemented them, those coming from the cascade).
We take all other hypo params (and associated sources) to be fixed, and modify only $\alpha_s$ (which scales the cascade energy and hence the luminosity of the scaling sources) until the maximum $\mathscr L$ (given the other parameters) is obtained.

To find the optimal $\alpha_s$, we can either numerically minimize

$$
-{\mathscr L} = 
    \Lambda
    -\sum_{d=1}^{D_{\rm hit}}
        \sum_{k_d = 1}^{K_d}
            q_{k_d} \ln
            \left(
                \lambda_d(t_{k_d}) + n_d
            \right)
$$

or solve (also numerically, since no analytic solution is possible)

$$
0 = 
    -\frac{\partial {\Lambda}}{\partial E_{\rm cscd}}
    +\sum_{d=1}^{D_{\rm hit}}
        \sum_{k_d = 1}^{K_d}
            \left(
                \frac{
                    q_{k_d}
                }{
                    \lambda_d(t_{k_d}) + n_d
                }
                \cdot
                \frac{\partial {\lambda_d(t_{k_d})}}{\partial E_{\rm cscd}}
            \right)
$$

although in the latter case it must be noted that a solution is not guaranteed to be a maximum of $\mathscr L$, only an extremum.

The latter case is cheaper to evaluate, though, so we'll look a little more closely at that.

$$
0 = - \frac{\partial {\Lambda}}{\partial E_{\rm cscd}}
    + \sum_{d=1}^{D_{\rm hit}}
        \sum_{k_d = 1}^{K_d}
            \left(
                \frac{
                    q_{k_d}
                }{
                    \lambda_d(t_{k_d}) + n_d
                }
                \cdot
                \frac{\partial {\lambda_d(t_{k_d})}}{\partial E_{\rm cscd}}
            \right)
$$

the change of cascade energy is represented by a change in $\alpha_s$, so the partials of $\Lambda$ and $\lambda_d$ are simply the respective factors of $\alpha_s$

$$
0 = - \sum_{i = 1}^{I_s} \ell^s_i \cdot {\rm TDI}(C_i)
    + \sum_{d=1}^{D_{\rm hit}}
        \sum_{k_d = 1}^{K_d}
            \left(
                \frac{
                    q_{k_d}
                }{
                    \lambda_d(t_{k_d}) + n_d
                }
                \cdot
                \sum_{i =1}^{I_s} \ell^s_i \cdot {\rm T}_d(C_i, t_{k_d})
            \right)
$$

writing out the full expression for $\lambda_d$ so it's clear where $\alpha_s$ (and the computational work) shows up,

$$
0 = - \sum_{i = 1}^{I_s} \ell^s_i \cdot {\rm TDI}(C_i)
    + \sum_{d=1}^{D_{\rm hit}} \sum_{k_d = 1}^{K_d}
        \frac{
            q_{k_d} \cdot \sum_{i =1}^{I_s} \ell^s_i \cdot {\rm T}_d(C_i, t_{k_d})
        }{
            n_d + \alpha_s \cdot \sum_{i =1}^{I_s} \ell^s_i \cdot {\rm T}_d(C_i, t_{k_d}) + \sum_{i = 1}^{I_p} \ell^p_i \cdot {\rm T}_d(C_i, t_{k_d}) + \sum_{i = 1}^{I_g} \ell^g_i \cdot {\rm T}_d(C_i, t_{k_d})
        }
$$

Writing in a form amenable to a generic solver (i.e., an expression that equals zero) and substituting the simpler variable names for the sums

$$
0 =
-\Lambda^s + \sum_{d=1}^{D_{\rm hit}} \sum_{k_d=1}^{K_d}
\frac{
    q_{k_d} \cdot \lambda_d^s(t_{k_d})
}{
    n_d + \alpha_s \cdot \lambda_d^s(t_{k_d}) + \lambda_d^p(t_{k_d}) + \lambda_d^g(t_{k_d})
}
$$

(where, again, the superscripts are _names_ and not _powers_).

This should be relatively cheap to solve numerically for $\alpha_s$ since all other quantities need only be computed once (the sum over pegleg sources can be stored from its evaluation in the pegleg loop and the sum over generic sources can be stored from its evaluation in the generic-optimizer loop).
We still do have to loop over all hits, though, to collate the current guess of $\alpha_s$ with the other quantities in the fraction.

## Final comments

The above derivation of $\mathscr L$ seems to be roughly corroborated by an example given in section 4 of the Barger paper cited above.
In that, extended maximum likelihood (EML) is used to estimate—from Poisson-generated events—an unknown (both intensity and shape) signal that is embedded in a known (both intensity and shape) background.

Note that if the above math holds, then we only need the TDI and time-dependent tables, and can dispense with the single-sensor time-independent tables.
The expression for $\mathscr L$ is also much simpler than the current one used, which could be a performance gain.